In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

In [ ]:
# Step 1: Data Loading
file_paths = ["Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv",
              "Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
              "Friday-WorkingHours-Morning.pcap_ISCX.csv",
              "Monday-WorkingHours.pcap_ISCX.csv",
              "Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
              "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
              "Tuesday-WorkingHours.pcap_ISCX.csv",
              "Wednesday-workingHours.pcap_ISCX.csv"]


In [ ]:
dfs = []
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate individual dataframes into one
combined_df = pd.concat(dfs)

In [ ]:
# Replace infinity or very large values with NaN
combined_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Check for missing values after replacing infinity or very large values
print(combined_df.isnull().sum())
# Drop rows with missing values
combined_df.dropna(inplace=True)
# Description of Data:
describe_info = combined_df.describe()

# Identification of All Zero Columns:
all_zeroes_cols = describe_info.loc[:,(describe_info.iloc[1:] == 0).all()]

# Dropping All Zero Columns:
combined_df.drop(columns=all_zeroes_cols, inplace=True)

# Checking Resulting DataFrame's Validity:
combined_df.shape  # check if resulting DataFrame valid



In [ ]:
# Check the data types of all columns
data_types = combined_df.dtypes
print(data_types)


In [ ]:
# Replace non-printable characters and strip leading/trailing spaces

combined_df.loc[:," Label"].replace({"Web Attack � XSS" : "XSS", "Web Attack � Sql Injection": "Sql Injection", "Web Attack � Brute Force": "Brute Force"}, inplace=True)
# Print unique values after cleaning
combined_df[' Label'].unique()



In [ ]:
combined_df

## Outliers

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot boxplots for numerical features
plt.figure(figsize=(16, 10))  # Increase figure size
sns.boxplot(data=combined_df)
plt.xticks(rotation=90)  # Rotate x-axis labels for better visibility

# Save the plot to a file with adjusted dimensions
plt.tight_layout()  # Adjust layout to fit labels
plt.savefig('outlier.png', dpi=300)  # Increase resolution if needed

plt.show()

# Calculate z-scores for each numerical feature
from scipy import stats

z_scores = stats.zscore(combined_df.select_dtypes(include=np.number))

# Threshold for identifying outliers (e.g., z-score > 3)
threshold = 3

# Create a boolean mask to identify outliers
outlier_mask = np.abs(z_scores) > threshold

# Filter out outliers from the DataFrame
cleaned_df = combined_df[~outlier_mask.any(axis=1)]


In [ ]:
cleaned_df

## Normalization

In [ ]:

xdf =cleaned_df.drop(' Label', axis=1)
ydf =cleaned_df[" Label"]

# Get the column names before MinMax normalization
column_names = xdf.columns

from tensorflow.keras import preprocessing
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

dense_array = xdf.astype('int16').values

# Now you can use 'dense_array' where dense data is required
scaler = MinMaxScaler()
# Apply MinMaxScaler directly to the numeric DataFrame
xdf = scaler.fit_transform(dense_array)


## Balancing

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Count occurrences of each label
label_counts = ydf.value_counts()

# Visualize label distribution
plt.figure(figsize=(8, 6))
ax = sns.barplot(x=label_counts.index, y=label_counts.values)
plt.title('Distribution of Labels')
plt.xlabel('Label')
plt.ylabel('Count')

# Rotate labels vertically
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='center')

# Save the figure
plt.savefig('before_balancing_label_distribution.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
ydf.unique()

In [ ]:

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# Print class distribution before balancing
class_distribution_before = ydf.value_counts()
print("Class Distribution before balancing:")
print(class_distribution_before)

# Remove classes with too few samples (for example, less than 5 samples)
min_samples_threshold = 5
filtered_classes = class_distribution_before[class_distribution_before >= min_samples_threshold].index
xdf_filtered = xdf[ydf.isin(filtered_classes)]
ydf_filtered = ydf[ydf.isin(filtered_classes)]

# Print the filtered class distribution
print("Class Distribution after filtering:")
print(ydf_filtered.value_counts())

# Determine the minimum number of samples across all remaining classes for under-sampling
min_samples = ydf_filtered.value_counts().min()

# Adjust the sampling_strategy for RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy={class_label: min_samples for class_label in ydf_filtered.unique()}, random_state=42)

# Create a reasonable sampling strategy for SMOTE based on existing classes
smote_sampling_strategy = {class_label: 10000 for class_label in ydf_filtered.unique()}

# Reduce the k_neighbors parameter for SMOTE to 1 for classes with less than 6 samples
smote = SMOTE(sampling_strategy=smote_sampling_strategy, k_neighbors=1, random_state=42)

# Define the pipeline
pipeline = Pipeline([('undersample', rus), ('oversample', smote)])

# Apply the pipeline to your filtered data
xdf_balanced, ydf_balanced = pipeline.fit_resample(xdf_filtered, ydf_filtered)

# Check the class distribution after balancing
class_distribution_after = ydf_balanced.value_counts()
print("\nClass Distribution after balancing:")
print(class_distribution_after)


In [ ]:
# Check the shape of the balanced dataset
print("Shape of balanced dataset:", xdf_balanced.shape, ydf_balanced.shape)


In [ ]:
# Check the shape of the balanced dataset
print("Shape of imbalanced dataset:", xdf.shape, ydf.shape)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Count occurrences of each label
label_counts = ydf_balanced.value_counts()

# Visualize label distribution
plt.figure(figsize=(8, 6))
sns.barplot(x=label_counts.index, y=label_counts.values)
plt.title('Distribution of Labels after SMOTE')
plt.xlabel('Label')
plt.ylabel('Count')

# Rotate x-axis labels vertically
plt.xticks(rotation=90)

# Save the figure
plt.savefig('after_balancing_label_distribution.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(xdf_balanced, ydf_balanced, test_size=0.3, random_state=42)


## Feature Selection RandomForest

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier()

# Perform k-fold cross-validation with k = 5
cv_scores = cross_val_score(rf_classifier, X_train, y_train, cv=5)

# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)


In [ ]:
# Fit the Random Forest classifier on the training data
rf_classifier.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Predict labels for the test set
y_pred = rf_classifier.predict(X_test)



In [ ]:
# Get feature importances
feature_importances = rf_classifier.feature_importances_

# Rank the importance of features
sorted_indices = np.argsort(feature_importances)[::-1]
sorted_features = column_names[sorted_indices]

# Print feature importance rankings
print("Feature Importance Rankings:")
for i, feature in enumerate(sorted_features):
    print(f"{i+1}. {feature}: {feature_importances[sorted_indices[i]]}")


In [ ]:
# Print selected feature names based on ranking
print("Selected Feature (RF) Names Based on Ranking:")
for i, feature_name in enumerate(sorted_features[:10], start=1):
    print(f"{i}. {feature_name}")


In [ ]:
import matplotlib.pyplot as plt

# Get the top 10 feature names and their importance values
top_features_rf_names = sorted_features[:10]
top_features_rf_importance = feature_importances[:10]

# Plot a pie chart for feature importance
plt.figure(figsize=(7, 6))
plt.pie(top_features_rf_importance, labels=top_features_rf_names, autopct='%1.1f%%')
plt.title('Feature Importance Ranking')

# Save the plot to a file
plt.savefig('Featur_Importance.png')

plt.show()


# Random Forest

In [ ]:
# Select the top 10 features based on importance

top_features = sorted_features[:10]

# Filter the training and testing data to include only the top features
X_train_top = X_train[:, sorted_indices[:10]]
X_test_top = X_test[:, sorted_indices[:10]]

# Initialize the Random Forest classifier
rf_classifier_top = RandomForestClassifier(random_state=42)

# Perform k-fold cross-validation with k = 5
cv_scores_top = cross_val_score(rf_classifier_top, X_train_top, y_train, cv=5)

# Print the cross-validation scores for the top features
print("Cross-validation scores with top 10 features:", cv_scores_top)

# Fit the Random Forest classifier on the training data with top features
rf_classifier_top.fit(X_train_top, y_train)

# Predict labels for the test set with top features
y_pred_top = rf_classifier_top.predict(X_test_top)

# Calculate accuracy score with top features
accuracy_top = accuracy_score(y_test, y_pred_top)
print("Accuracy with top 10 features:", accuracy_top)

# Generate classification report with top features
print("Classification Report with top 10 features:")
print(classification_report(y_test, y_pred_top))


In [ ]:
from sklearn.metrics import classification_report

# Generate classification report with top features
report = classification_report(y_test, y_pred_top, target_names=rf_classifier_top.classes_, output_dict=True)

# Print total accuracy, precision, recall, and F1-score
print("Total Accuracy:", report['accuracy'])
print("Total Precision:", report['macro avg']['precision'])
print("Total Recall:", report['macro avg']['recall'])
print("Total F1-Score:", report['macro avg']['f1-score'])


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate confusion matrix for Random Forest
conf_matrix_rf = confusion_matrix(y_test, y_pred_top)

# Plot confusion matrix for Random Forest
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_rf, annot=True, fmt='d', cmap='Blues', 
            xticklabels=rf_classifier_top.classes_, 
            yticklabels=rf_classifier_top.classes_)
plt.title('Random Forest')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()

# Save the plot for Random Forest
plt.savefig('RF_confusion_matrix.png')

# Show plot for Random Forest
plt.show()


In [ ]:
# Select the index of the instance you want to check
instance_index = 2  # Change this index to the instance you want to check

# Get the features and actual label of the instance
instance_features = X_test_top[instance_index].reshape(1, -1)
actual_label = y_test.iloc[instance_index]

# Predict the label for the instance using the trained model
predicted_label = rf_classifier_top.predict(instance_features)[0]

# Check if the predicted label matches the actual label
prediction_correct = predicted_label == actual_label

# Print the result
if prediction_correct:
    print("Prediction is correct for instance", instance_index)
    print("Actual label:", actual_label)
    print("Predicted label:", predicted_label)
else:
    print("Prediction is incorrect for instance", instance_index)
    print("Actual label:", actual_label)
    print("Predicted label:", predicted_label)


In [ ]:
import lime
import lime.lime_tabular
# Initialize the explainer
explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train_top,
    feature_names=top_features,
    class_names=['BENIGN', 'DDOS'],  # Adjust the class names as per your labels
    discretize_continuous=True
)
observation_id = 1  # Adjust this to the specific ID you want to explain
observation = X_test_top[observation_id].reshape(1, -1)
# Explain the model's prediction on the selected instance
explanation = explainer.explain_instance(
    observation.flatten(),  # The instance to explain
    rf_classifier_top.predict_proba,  # The prediction function
    num_features=10  # Number of features to show in explanation
)
# Show the explanation in the notebook
explanation.show_in_notebook(show_table=True, show_all=False)


# DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Initialize the Decision Tree classifier
dt_classifier = DecisionTreeClassifier()

# Train the Decision Tree classifier on the training data
dt_classifier.fit(X_train_top, y_train)

# Predict labels for the test set using the trained Decision Tree classifier
y_pred_dt = dt_classifier.predict(X_test_top)

# Calculate accuracy score
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Accuracy with Decision Tree:", accuracy_dt)

# Generate classification report for the Decision Tree classifier
print("Classification Report for Decision Tree:")
print(classification_report(y_test, y_pred_dt))


In [ ]:
from sklearn.metrics import classification_report

# Generate classification report with top features
dt_report = classification_report(y_test, y_pred_dt, target_names=dt_classifier.classes_, output_dict=True)

# Print total accuracy, precision, recall, and F1-score
print("Total Accuracy:", dt_report['accuracy'])
print("Total Precision:", dt_report['macro avg']['precision'])
print("Total Recall:", dt_report['macro avg']['recall'])
print("Total F1-Score:", dt_report['macro avg']['f1-score'])


In [ ]:
import numpy as np

# Select the index of the instance you want to check
dt_instance_index = 0  # Change this index to the instance you want to check

# Get the features and actual label of the instance
dt_instance_features = np.expand_dims(X_test_top[dt_instance_index], axis=0)
dt_actual_label = y_test.iloc[dt_instance_index]

# Predict the label for the instance using the trained model
dt_predicted_label = dt_classifier.predict(dt_instance_features)[0]

# Check if the predicted label matches the actual label
dt_prediction_correct = dt_predicted_label == dt_actual_label

# Print the result
if dt_prediction_correct:
    print("Prediction is correct for instance", dt_instance_index)
    print("Actual label:", dt_actual_label)
    print("Predicted label:", dt_predicted_label)
else:
    print("Prediction is incorrect for instance", dt_instance_index)
    print("Actual label:", dt_actual_label)
    print("Predicted label:", dt_predicted_label)


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Get unique class labels
classes = np.unique(y_test)

# Calculate confusion matrix for Decision Tree
conf_matrix_dt = confusion_matrix(y_test, y_pred_dt)

# Plot confusion matrix for Decision Tree
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_dt, annot=True, fmt='d', cmap='Blues', 
            xticklabels=classes, 
            yticklabels=classes)
plt.title('Decision Tree')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()

# Save the plot for Decision Tree
plt.savefig('DT_confusion_matrix.png')

# Show plot for Decision Tree
plt.show()


In [ ]:
import lime
import lime.lime_tabular

# Initialize the LIME explainer for Decision Tree
dt_explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train_top,  # Training data with top features for Decision Tree
    feature_names=top_features,  # Top feature names for Decision Tree
    class_names=['BENIGN', 'DDOS'],  # Class names (adjust as per your labels)
    discretize_continuous=True  # Discretize continuous features
)

# Select the observation ID you want to explain for Decision Tree
dt_observation_id = 1  # Adjust this to the specific ID you want to explain

# Get the observation to explain for Decision Tree
dt_observation = X_test_top[dt_observation_id].reshape(1, -1)

# Explain the model's prediction on the selected instance for Decision Tree
dt_explanation = dt_explainer.explain_instance(
    dt_observation.flatten(),  # Instance to explain
    dt_classifier.predict_proba,  # Prediction function (Decision Tree)
    num_features=10  # Number of features to show in explanation
)

# Show the explanation in the notebook for Decision Tree
dt_explanation.show_in_notebook(show_table=True, show_all=False)



# SVM classifier

In [ ]:
from sklearn.svm import SVC

# Initialize the SVM classifier
svm_classifier = SVC(kernel='linear', C=1.0, probability=True)  # You can adjust kernel and C parameter as needed

# Train the SVM classifier on the training data
svm_classifier.fit(X_train_top, y_train)

# Predict labels for the test set using the trained SVM classifier
y_pred_svm = svm_classifier.predict(X_test_top)

# Calculate accuracy score
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("Accuracy with SVM:", accuracy_svm)

# Generate classification report for the SVM classifier
print("Classification Report for SVM:")
print(classification_report(y_test, y_pred_svm))


In [ ]:
# Select the index of the instance you want to check
svm_instance_index = 70  # Change this index to the instance you want to check

# Get the features and actual label of the instance
svm_instance_features = X_test_top[svm_instance_index].reshape(1, -1)
svm_actual_label = y_test.iloc[svm_instance_index]

# Predict the label for the instance using the trained model
svm_predicted_label = svm_classifier.predict(svm_instance_features)[0]

# Check if the predicted label matches the actual label
svm_prediction_correct = svm_predicted_label == svm_actual_label

# Print the result
if svm_prediction_correct:
    print("Prediction is correct for instance", svm_instance_index)
    print("Actual label:", svm_actual_label)
    print("Predicted label:", svm_predicted_label)
else:
    print("Prediction is incorrect for instance", svm_instance_index)
    print("Actual label:", svm_actual_label)
    print("Predicted label:", svm_predicted_label)


In [ ]:
from sklearn.metrics import classification_report

# Generate classification report with top features
svm_report = classification_report(y_test, y_pred_svm, target_names=svm_classifier.classes_, output_dict=True)

# Print total accuracy, precision, recall, and F1-score
print("Total Accuracy:",svm_report['accuracy'])
print("Total Precision:", svm_report['macro avg']['precision'])
print("Total Recall:", svm_report['macro avg']['recall'])
print("Total F1-Score:", svm_report['macro avg']['f1-score'])


In [ ]:
from sklearn.metrics import confusion_matrix

# Calculate confusion matrix for SVM
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)

# Plot confusion matrix for SVM
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_svm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=svm_classifier.classes_, 
            yticklabels=svm_classifier.classes_)
plt.title('SVM')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()

# Save the plot for SVM
plt.savefig('SVM_confusion_matrix.png')

# Show plot for SVM
plt.show()


In [ ]:
import lime
import lime.lime_tabular

# Initialize the LIME explainer for SVM
svm_explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train_top,  # Training data with top features for SVM
    feature_names=top_features,  # Top feature names for SVM
    class_names=['BENIGN', 'DDOS'],  # Class names (adjust as per your labels)
    discretize_continuous=True  # Discretize continuous features
)

# Select the observation ID you want to explain for SVM
svm_observation_id = 1  # Adjust this to the specific ID you want to explain

# Get the observation to explain for SVM
svm_observation = X_test_top[svm_observation_id].reshape(1, -1)

# Explain the model's prediction on the selected instance for SVM
svm_explanation = svm_explainer.explain_instance(
    svm_observation.flatten(),  # Instance to explain
    svm_classifier.predict_proba,  # Prediction function (SVM)
    num_features=10  # Number of features to show in explanation
)

# Show the explanation in the notebook for SVM
svm_explanation.show_in_notebook(show_table=True, show_all=False)



# VotingClassifier

In [ ]:
from sklearn.ensemble import VotingClassifier

# Define the individual classifiers
dt_classifier = DecisionTreeClassifier()
rf_classifier_top = RandomForestClassifier(random_state=42)
svm_classifier = SVC(kernel='linear', C=1.0, probability=True)  # Enable probability estimation for SVM

# List of (name, estimator) tuples for VotingClassifier
estimators = [
    ('decision_tree', dt_classifier),
    ('random_forest', rf_classifier_top ),
    ('svm', svm_classifier)
]

# Initialize the VotingClassifier
voting_classifier = VotingClassifier(estimators, voting='soft')  # You can adjust voting method as needed

# Train the VotingClassifier on the training data
voting_classifier.fit(X_train_top, y_train)

# Predict labels for the test set using the trained VotingClassifier
y_pred_voting = voting_classifier.predict(X_test_top)

# Calculate accuracy score
accuracy_voting = accuracy_score(y_test, y_pred_voting)
print("Accuracy with Voting Classifier:", accuracy_voting)

# Generate classification report for the VotingClassifier
print("Classification Report for Voting Classifier:")
print(classification_report(y_test, y_pred_voting))


In [ ]:
from sklearn.metrics import classification_report

# Generate classification report with top features
voting_report = classification_report(y_test, y_pred_voting, target_names=voting_classifier.classes_, output_dict=True)

# Print total accuracy, precision, recall, and F1-score
print("Total Accuracy:",voting_report['accuracy'])
print("Total Precision:", voting_report['macro avg']['precision'])
print("Total Recall:", voting_report['macro avg']['recall'])
print("Total F1-Score:", voting_report['macro avg']['f1-score'])


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate confusion matrix for Voting Classifier
conf_matrix_voting = confusion_matrix(y_test, y_pred_voting)

# Plot confusion matrix for Voting Classifier
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_voting, annot=True, fmt='d', cmap='Blues', 
            xticklabels=voting_classifier.classes_, 
            yticklabels=voting_classifier.classes_)
plt.title('Voting Classifier')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()

# Save the plot for Voting Classifier
plt.savefig('Voting_confusion_matrix.png')

# Show plot for Voting Classifier
plt.show()


In [ ]:
import lime
import lime.lime_tabular

# Initialize the LIME explainer for SVM
voting_explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train_top,  # Training data with top features for SVM
    feature_names=top_features,  # Top feature names for SVM
    class_names=['BENIGN', 'DDOS'],  # Class names (adjust as per your labels)
    discretize_continuous=True  # Discretize continuous features
)

# Select the observation ID you want to explain for SVM
voting_observation_id = 1  # Adjust this to the specific ID you want to explain

# Get the observation to explain for voting
voting_observation = X_test_top[voting_observation_id].reshape(1, -1)

# Explain the model's prediction on the selected instance for SVM
voting_explanation = voting_explainer.explain_instance(
   voting_observation.flatten(),  # Instance to explain
    voting_classifier.predict_proba,  # Prediction function (SVM)
    num_features=10  # Number of features to show in explanation
)

# Show the explanation in the notebook for voting
voting_explanation.show_in_notebook(show_table=True, show_all=False)



In [ ]:
voting_explanation.as_list()